# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

Please make sure if you are running this notebook in the workspace that you have chosen GPU rather than CPU mode.

In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import numpy as np
import torch
from torch import nn
from torch import optim
import seaborn as sb
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
from PIL import Image

In [2]:
import signal

from contextlib import contextmanager

import requests


DELAY = INTERVAL = 4 * 60  # interval time in seconds
MIN_DELAY = MIN_INTERVAL = 2 * 60
KEEPALIVE_URL = "https://nebula.udacity.com/api/v1/remote/keep-alive"
TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
TOKEN_HEADERS = {"Metadata-Flavor":"Google"}


def _request_handler(headers):
    def _handler(signum, frame):
        requests.request("POST", KEEPALIVE_URL, headers=headers)
    return _handler


@contextmanager
def active_session(delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import active session

    with active_session():
        # do long-running work here
    """
    token = requests.request("GET", TOKEN_URL, headers=TOKEN_HEADERS).text
    headers = {'Authorization': "STAR " + token}
    delay = max(delay, MIN_DELAY)
    interval = max(interval, MIN_INTERVAL)
    original_handler = signal.getsignal(signal.SIGALRM)
    try:
        signal.signal(signal.SIGALRM, _request_handler(headers))
        signal.setitimer(signal.ITIMER_REAL, delay, interval)
        yield
    finally:
        signal.signal(signal.SIGALRM, original_handler)
        signal.setitimer(signal.ITIMER_REAL, 0)


def keep_awake(iterable, delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import keep_awake

    for i in keep_awake(range(5)):
        # do iteration with lots of work here
    """
    with active_session(delay, interval): yield from iterable


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [3]:
data_dir = 'flowers'
batch_size = 32

#Transfomers - more than meets the eye!
train_transformer = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

valid_transformer = transforms.Compose([transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

test_transformer = transforms.Compose([transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

#Datasets
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

train_data = datasets.ImageFolder(train_dir, transform=train_transformer)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transformer)
test_data = datasets.ImageFolder(test_dir, transform=test_transformer)

#Dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

In [ ]:
model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
 94%|█████████▍| 520552448/553433881 [00:05<00:00, 96047405.08it/s]

In [ ]:
def validation(model, loader, criterion):
    loader_loss = 0
    accuracy = 0
    step = 0
    float_tensor = torch.FloatTensor if device == 'cpu' else torch.cuda.FloatTensor
    for images, labels in loader:
        step += 1
        images, labels = images.to(device), labels.to(device)
        output = model.forward(images)
        loader_loss += criterion(output, labels).item()
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        #Convert equality to Float Tensor so you can take the mean.
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return loader_loss, accuracy

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
#Match up inputs, create classifier with final 102 outputs for 102 flowers
#in dataset

fred = nn.Sequential(OrderedDict([
    ('layer1', nn.Linear(25088, 5000)),
    ('drop1', nn.Dropout(p=0.2)),
    ('relu1', nn.ReLU()),
    ('layer2', nn.Linear(5000, 1000)),
    ('drop2', nn.Dropout(p=0.2)),
    ('relu2', nn.ReLU()),
    ('layer3', nn.Linear(1000, 102)),
    ('ouput', nn.LogSoftmax(dim=1))
    ]))
    
model.classifier = fred

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9, nesterov=True)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
def train_network(e, print_every):
    with active_session(): 
        epochs = e
        print_every = print_every
        steps = 0
        #device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

        model.to(device)
        #DON"T FORGET TO PUT THE MODEL IN TRAINING MODE!
        model.train()

        for epoch in range(epochs):
            running_loss = 0
            for ii, (images, labels) in enumerate(train_loader):
                steps += 1
        
                images, labels = images.to(device), labels.to(device)
        
                optimizer.zero_grad()
        
                outputs = model.forward(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        
                running_loss += loss.item()
        
                if steps % print_every == 0:
                    model.eval()
            
                    # Turn off gradients for validation, saves memory and computations
                    with torch.no_grad():
                        loader_loss, accuracy = validation(model, valid_loader, criterion)
                
                    print("Epoch: {}/{}.. ".format(epoch+1, epochs),
                      "Training Loss: {:.4f}.. ".format(running_loss/print_every),
                      "Valid Loss: {:.4f}.. ".format(loader_loss/len(valid_loader)),
                      "Valid Accuracy: {:.4f}".format(accuracy/len(valid_loader)))
            
                running_loss = 0
            
                # Make sure training is back on
                model.train()
            
train_network(1, 40)

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
#Testing
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
correct = 0
total = 0
with torch.no_grad():
    for ii, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
print('Fred the model: \n\n', model, '\n')
print('The state dict keys: \n\n', model.state_dict().keys())

In [ ]:
def save_model_and_checkpoint():
    #torch.save(model, 'model.pth') Not necessary because we only need to 
    #save the classifier, not the whole model?
    #model.class_to_idx = train_data.class_to_idx
    
    checkpoint = torch.save({
        'classifier': fred,
        'arch': 'vgg16',
        'class_to_idx': train_data.class_to_idx,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()},
        'checkpoint.pth')

save_model_and_checkpoint()

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# If I understand correctly, the below is a more refined way to do this than
# the tutorial on the pytorch site. 
#def load_model(PATH): 
#    model = torch.load(PATH)
#   return model

def load_checkpoint(PATH):
    checkpoint = torch.load(PATH)
    if checkpoint['arch'] == 'vgg16':
        model = models.vgg16(pretrained=True)
    model.classifier = checkpoint['classifier']
    model.class_to_idx = checkpoint['class_to_idx']
    class_to_idx = model.class_to_idx
    model.load_state_dict(checkpoint['state_dict'], strict = False)
    optimizer = checkpoint['optimizer']
    
    for param in model.parameters():
        param.requires_grad = False
    
    return model, class_to_idx

model, class_to_idx = load_checkpoint('checkpoint.pth')

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(img):
    width, height = img.size
    #print('Printing old height and width')
    #print(width, height)
    n_size = ((256, 256))
    img.thumbnail(n_size)
    n_width, n_height = img.size
    #print('\n Printing new width and new height')
    #print(n_width, n_height)
    
    left = (n_width - 224)/2
    #print('\n Printing left: ' + str(left))
    top = (n_height - 224)/2
    right = (n_width + 224)/2
    bottom = (n_height + 224)/2
    
    img = img.crop((left, top, right, bottom)) #This is a lazy method, you gotta assign it!!!
    #print('\nPrinting cropped image size')
    #print(img.size)
    
    #Normalize
    img = np.array(img)
    img = img / 235
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225]) 
    #img is already an nd.array

    img = (img - mean)/std 
    img = img.transpose((2, 0, 1))
    #print(img.shape)
    return img

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

image = Image.open('flowers/test/1/image_06760.jpg')
imshow(process_image(image))


## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, class_idx):
    
    #set model to evaluation mode
    model.eval() 
    #keep defining this inside functions, should move to global?
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    
    image = Image.open(image_path)
    input_image = process_image(image)
    input_image = torch.FloatTensor(np.array(input_image))
    input_image = input_image.unsqueeze(0)
    
    print(input_image)
    
    with torch.no_grad():
        input_image = input_image.to(device)
        predict = model.forward(input_image)
        predict = torch.exp(predict)
        probs, classes = predict.topk(k=5)
        probs = probs.cpu().numpy()[0]

        #probs = [model.classifier.class_to_idx[str(i)] for i in classes]

        classes = classes.cpu().numpy()[0]
        
        #classes = [model.classifier.class_to_idx[str(i)] for i in classes]#class_to_idx = class_to_idx
        #idx_to_class = {str(value):int(key) for key, value in class_idx.items()}
        #probs = probs.numpy()
        #classes = np.array([idx_to_class[idx] for idx in classes.numpy()])
        
        
        
        #top_k_names = list()

        #for idx in classes:

        #    top_k_names.extend([v for k, v in cat_to_name.items() if k == idx])

    return probs, classes

probs, classes = predict('flowers/test/1/image_06760.jpg', model, class_to_idx)

print(probs)
print(classes)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
print(cat_to_name)

In [ ]:
def class_idx_to_name(classes):
    class_names = []
    for key, value in cat_to_name.items():
        if key in classes:
            class_names.append(value)
    return class_names

print(class_idx_to_name(classes))

In [ ]:
def san_check(image_path, model, class_idx):
    image_path = image_path
    predict(image_path, model, class_idx)
    image = Image.open(image_path)
    imshow(process_image(image))
    ax = sb.countplot(x = probs)
    
san_check('flowers/test/1/image_06760.jpg', model, class_to_idx)